In [ ]:
# Step 1: Import Libraries
import tensorflow as tf
import numpy as np
import os
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from google.colab import files

# Step 2: Clone Dataset from Your GitHub Repository
!git clone https://github.com/ujjwalr03/image-classification-of-animals.git
DATASET_PATH = "image-classification-of-animals/dataset/"

# Step 3: Data Preprocessing
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_data = datagen.flow_from_directory(DATASET_PATH, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode="categorical", subset="training")
val_data = datagen.flow_from_directory(DATASET_PATH, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode="categorical", subset="validation")

# Step 4: Load MobileNetV2 (Transfer Learning)
base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights="imagenet")
base_model.trainable = False
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(128, activation="relu")(x)
x = Dropout(0.3)(x)
output = Dense(train_data.num_classes, activation="softmax")(x)

# Step 5: Compile and Train Model
model = Model(inputs=base_model.input, outputs=output)
model.compile(optimizer=Adam(learning_rate=0.001), loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(train_data, validation_data=val_data, epochs=10)

# Step 6: Save Model
model.save("animal_classifier.h5")

# Step 7: Upload an Image for Testing
uploaded = files.upload()
image_path = list(uploaded.keys())[0]

# Step 8: Load Model for Prediction
model = tf.keras.models.load_model("animal_classifier.h5")

# Step 9: Predict on Uploaded Image
def preprocess_image(img_path):
    img = cv2.imread(img_path)
    img = cv2.resize(img, (224, 224))
    img = img / 255.0
    img = np.expand_dims(img, axis=0)
    return img

class_labels = os.listdir(DATASET_PATH)

predicted_class = class_labels[np.argmax(model.predict(preprocess_image(image_path)))]
print(f"{predicted_class}")